# Compare Example Beams Across Versions
---
Generate Current Devel Example Beams

In [1]:
from distgen import Generator

import glob
from pathlib import Path
import numpy as np
from pmd_beamphysics import ParticleGroup

In [2]:
from distgen import __version__

In [3]:
__version__

'2.0.0+1.g25ab4c2.dirty'

## Generate Archive

In [4]:
YAMLS = glob.glob('../examples/data/*yaml')

In [5]:
for YAML in YAMLS:

    D = Generator(YAML)
    D['n_particle']=1_000
    P = D.run()
    P.write(f'archive/{Path(YAML).name.replace('.yaml', '.h5')}') 


In [6]:
import os

In [7]:
os.getcwd()

'/Users/colwyngulliford/GitHub/distgen/docs/regression_tests'

## Compare Archive To A Previous Archive

In [8]:
#PREVIOUS_ARCHIVE = 'archive'

PREVIOUS_ARCHIVE = '/Users/colwyngulliford/devel/distgen/distgen2.0.0'

def test_beam(beam_file, previous_archive, abs_tol = 1e-9, rel_tol = 1e-11, verbose=True):

    beam_file = Path(beam_file)
    name = beam_file.name.replace('.yaml', '')
    p1file = f'{previous_archive}/{name}.h5'
    p2file = f'archive/{name}.h5'

    if(Path(p1file).exists() and Path(p2file).exists()):
        if(verbose):
            print('Checking:', Path(p1file).name, 'against', p2file)
        
        P1, P2 = ParticleGroup(p1file), ParticleGroup(p2file)
        
        vs = ['x', 'y', 'z', 'px', 'py', 'pz', 'weight']
        for v in vs:
            abs_deviation = P1[v] - P2[v]
            assert np.max(np.abs(abs_deviation)) < abs_tol, f'Max[ |absolute deviation| ] for {v}: {np.max(np.abs(abs_deviation))} for {Path(p1file).name}'
            non_zero = np.abs(P2[v])>0
            
            if(np.any(non_zero)):
                rel_deviation = (P1[v][non_zero] - P2[v][non_zero])/P2[v][non_zero]
                assert np.max(np.abs(rel_deviation)) < rel_tol, f'Max[ |relative deviation| ] for {v}: {np.max(np.abs(rel_deviation))} for {Path(p1file).name}'

        

def test_beams(beam_files, previous_archive, verbose=True, abs_tol=1e-10, rel_tol=1e-12):

    print(f'Comparing the current beam archive to {previous_archive}\n')

    for beam_file in beam_files:
        test_beam(beam_file, previous_archive, verbose=verbose, abs_tol = abs_tol, rel_tol = rel_tol)

    if(verbose):
        print('\nAll beams check to w/absolute tolerance of:', abs_tol )
        print('All beams check to an w/relative tolerance of:', rel_tol )

test_beams(YAMLS, PREVIOUS_ARCHIVE, verbose=True, abs_tol = 1e-11, rel_tol=1e-12)

Comparing the current beam archive to /Users/colwyngulliford/devel/distgen/distgen2.0.0


All beams check to w/absolute tolerance of: 1e-11
All beams check to an w/relative tolerance of: 1e-12
